In [7]:
#currently not being used.
import yfinance as yf
import pandas as pd
import requests
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [8]:
gold_data = yf.download('GLD', start='2005-05-08', end='2024-05-20',interval='1mo')
print(gold_data.head())

[*********************100%%**********************]  1 of 1 completed

                 Open       High        Low      Close  Adj Close    Volume
Date                                                                       
2005-06-01  41.410000  44.130001  41.400002  43.439999  43.439999  36145800
2005-07-01  43.400002  43.400002  41.700001  42.820000  42.820000  33962100
2005-08-01  43.080002  44.720001  42.810001  43.400002  43.400002  32270600
2005-09-01  43.930000  47.230000  43.849998  46.700001  46.700001  44606600
2005-10-01  46.619999  47.720001  45.830002  46.400002  46.400002  45491800


In [9]:
gold_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 228 entries, 2005-06-01 to 2024-05-01
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       228 non-null    float64
 1   High       228 non-null    float64
 2   Low        228 non-null    float64
 3   Close      228 non-null    float64
 4   Adj Close  228 non-null    float64
 5   Volume     228 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 12.5 KB


In [10]:
# Handle missing values
gold_data = gold_data.dropna()

# Create relevant features
gold_data['MA_2'] = gold_data['Close'].rolling(window=2).mean()
gold_data['MA_5'] = gold_data['Close'].rolling(window=5).mean()
gold_data['Volatility'] = gold_data['Close'].rolling(window=2).std()
gold_data['Return'] = gold_data['Close'].pct_change()

In [11]:
gold_data = gold_data.dropna()
print(gold_data.head())

# Define features and target
features = ['MA_2', 'MA_5', 'Volatility', 'Return']
X = gold_data[features]
y = gold_data['Close']

                 Open       High        Low      Close  Adj Close     Volume  \
Date                                                                           
2005-10-01  46.619999  47.720001  45.830002  46.400002  46.400002   45491800   
2005-11-01  46.070000  49.990002  45.320000  49.099998  49.099998   37083000   
2005-12-01  49.630001  53.759998  48.799999  51.580002  51.580002   79411300   
2006-01-01  52.290001  57.049999  52.099998  56.700001  56.700001  103662000   
2006-02-01  56.790001  57.299999  53.330002  56.070000  56.070000   86851300   

                 MA_2       MA_5  Volatility    Return  
Date                                                    
2005-10-01  46.550001  44.552000    0.212131 -0.006424  
2005-11-01  47.750000  45.684000    1.909186  0.058190  
2005-12-01  50.340000  47.436001    1.753627  0.050509  
2006-01-01  54.140001  50.096001    3.620386  0.099263  
2006-02-01  56.385000  51.970000    0.445478 -0.011111  


In [12]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
# Train a regression model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 8.158072302250815


In [ ]:
import pickle 
with open('gold_model.pkl','wb') as file:
    pickle.dump(model,file)

In [ ]:
with open('gold_scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)